# Proyecto TA

In [28]:
import pandas as pd 
import numpy as np
import ta 
from ta.trend import MACD
from ta.momentum import RSIIndicator 
from typing import List
from scipy.optimize import minimize

In [13]:
# Data
train_d = pd.read_csv('Files/aapl_5m_train.csv')
validation_d = pd.read_csv('Files/aapl_5m_validation.csv')

In [14]:
#Signal generation 
def gen_signals(strat:List[int],signal_type:str,*args):
    ct = 0
    strat_args = args
    macd,sma,rsi = strat
    df = pd.DataFrame()
    
    if sma:
        long = strat_args[ct]
        short = strat_args[ct+1]
        ct += 2
        
        short_sma = ta.trend.SMAIndicator(train_d.Close,window=long) 
        long_sma = ta.trend.SMAIndicator(train_d.Close,window=short)
        
        train_d["short_sma"] = short_sma.sma_indicator()
        train_d["long_sma"] = long_sma.sma_indicator()    
    
    if macd:
        slow_period = 26 
        fast_period = 12
        signal_period = 9
        
        macd = MACD(train_d['Close'], window_slow=slow_period, window_fast=fast_period, window_sign=signal_period)
        train_d['MACD'] = macd.macd()
        train_d['Signal'] = macd.macd_signal()
        
    if rsi:
        rsi_period = strat_args[ct]
        ct += 1
        
        rsi_indicator = RSIIndicator(train_d['Close'],window=rsi_period)
        train_d['RSI'] = rsi_indicator.rsi()

    
    train_d['SMA_Buy_Signal'] = 0
    train_d['MACD_Buy_Signal'] = 0
    train_d['RSI_Buy_Signal'] = 0
    train_d['SMA_Sell_Signal'] = 0
    train_d['MACD_Sell_Signal'] = 0
    train_d['RSI_Sell_Signal'] = 0
    
    #Generar los BUY/SELL dependiendo del indicador
    if signal_type == "BUY":
        
        #train_d['SMA_Buy_Signal'] = 0
        train_d.loc[train_d['long_sma'] > train_d['short_sma'], 'SMA_Buy_Signal'] = 1

        #train_d['MACD_Buy_Signal'] = 0
        train_d.loc[(train_d['MACD'] > train_d['Signal']) & (train_d['MACD'] > 0), 'MACD_Buy_Signal'] = 1

        #train_d['RSI_Buy_Signal'] = 0
        train_d.loc[train_d['RSI'] > 70, 'RSI_Buy_Signal'] = 1

    elif signal_type == "SELL":
        
        #train_d['SMA_Sell_Signal'] = 0
        train_d.loc[train_d['long_sma'] < train_d['short_sma'], 'SMA_Sell_Signal'] = 1

        #train_d['MACD_Sell_Signal'] = 0
        train_d.loc[(train_d['MACD'] < train_d['Signal']) & (train_d['MACD'] < 0), 'MACD_Sell_Signal'] = 1

        #train_d['RSI_Sell_Signal'] = 0
        train_d.loc[train_d['RSI'] < 30, 'RSI_Sell_Signal'] = 1

    return train_d   

In [15]:
strategy = [1,1,1]  # 1 MACD, 1 SMA, 1 RSI
signal_type =  "SELL", "BUY"  
sma_args = [50, 20]  # Long and Short SMA periods
rsi_args = [14]  # RSI period

result_df = gen_signals(strategy, signal_type, *sma_args, *rsi_args) 
result_df

,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume,short_sma,long_sma,MACD,Signal,RSI,SMA_Buy_Signal,MACD_Buy_Signal,RSI_Buy_Signal,SMA_Sell_Signal,MACD_Sell_Signal,RSI_Sell_Signal
0,1641220200,0,2022-01-03 14:30:00,177.830001,179.309997,177.710006,178.964996,4441780.0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0
1,1641220500,0,2022-01-03 14:35:00,178.970001,180.399993,178.919998,180.330001,2482136.0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0
2,1641220800,0,2022-01-03 14:40:00,180.330001,180.839996,180.210006,180.595001,2219825.0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0
3,1641221100,0,2022-01-03 14:45:00,180.600006,180.750000,179.470001,179.940002,2018163.0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0
4,1641221400,0,2022-01-03 14:50:00,179.929992,180.229904,179.399993,180.210006,1944837.0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19788,1672432800,0,2022-12-30 20:40:00,129.029998,129.130004,128.919998,129.054992,833767.0,128.618758,128.515600,0.151197,0.066889,66.263680,0,0,0,0,0,0
19789,1672433100,0,2022-12-30 20:45:00,129.059997,129.399993,129.020004,129.380004,2136910.0,128.633958,128.579600,0.186237,0.090758,73.183295,0,0,0,0,0,0
19790,1672433400,0,2022-12-30 20:50:00,129.375000,129.929992,129.330001,129.764404,2879267.0,128.658046,128.660325,0.242232,0.121053,78.737959,0,0,0,0,0,0
19791,1672433700,0,2022-12-30 20:55:00,129.764999,129.949996,129.619995,129.949996,3896812.0,128.683946,128.754325,0.298147,0.156472,80.805224,0,0,0,0,0,0


In [16]:
# Combinations (Strategies)
n = list(range(1, 2**5))
combinations = list(map(lambda x: [int(bit) for bit in f"{x:05b}"], n))

In [ ]:
# Backtesting
comission = .025
stop_loss = .035
take_profit = .035
cash = 1000000
positions = []
operations = []
trading_data = train_d

class Backtesting:
    def __init__(self, trading_data, comission, stop_loss, take_profit):
        self.trading_data = trading_data
        self.comission = comission
        self.stop_loss = stop_loss
        self.take_profit = take_profit
        self.cash = 1000000  # Initial cash balance
        self.positions = []

    class Order:
        def __init__(self, timestamp, bought_at, stop_loss,
                     take_profit, order_type, sold_at=None,
                     is_active=True):
            self.timestamp = timestamp
            self.bought_at = bought_at
            self.sold_at = sold_at
            self.stop_loss = stop_loss
            self.take_profit = take_profit
            self.order_type = order_type
            self.is_active = is_active

    def backtest(self):
        for i, row in self.trading_data.iterrows():
            # Close positions
            price = row['Close']
            for position in self.positions:
                if position.is_active:
                    if price <= position.stop_loss:
                        self.cash += price * (1 - self.comission)
                        position.is_active = False
                        position.sold_at = price
                    elif price >= position.take_profit:
                        self.cash += price * (1 - self.comission)
                        position.is_active = False
                        position.sold_at = price

            # Buy SMA
            if row['SMA_Buy_Signal']:
                if self.cash >= row['Close'] * (1 + self.comission):
                    self.cash -= row['Close'] * (1 + self.comission)
                    order = self.Order(timestamp=row['Timestamp'],
                                       bought_at=row['Close'],
                                       stop_loss=row['Close'] * (1 - self.stop_loss),
                                       take_profit=row['Close'] * (1 + self.take_profit),
                                       order_type="LONG")
                    self.positions.append(order)

            # Buy MACD
            if row['MACD_Buy_Signal']:
                if self.cash >= row['Close'] * (1 + self.comission):
                    self.cash -= row['Close'] * (1 + self.comission)
                    order = self.Order(timestamp=row['Timestamp'],
                                       bought_at=row['Close'],
                                       stop_loss=row['Close'] * (1 - self.stop_loss),
                                       take_profit=row['Close'] * (1 + self.take_profit),
                                       order_type="LONG")
                    self.positions.append(order)

            # Buy RSI
            if row['RSI_Buy_Signal']:
                if self.cash >= row['Close'] * (1 + self.comission):
                    self.cash -= row['Close'] * (1 + self.comission)
                    order = self.Order(timestamp=row['Timestamp'],
                                       bought_at=row['Close'],
                                       stop_loss=row['Close'] * (1 - self.stop_loss),
                                       take_profit=row['Close'] * (1 + self.take_profit),
                                       order_type="LONG")
                    self.positions.append(order)

            # Sell SMA
            if row['SMA_Sell_Signal']:
                for position in self.positions:
                    if position.is_active and position.order_type == "LONG":
                        if row['Close'] * (1 - self.comission) >= position.take_profit:
                            self.cash += row['Close'] * (1 - self.comission)
                            position.is_active = False
                            position.sold_at = row['Close']
                        elif row['Close'] * (1 + self.comission) <= position.stop_loss:
                            self.cash += row['Close'] * (1 - self.comission)
                            position.is_active = False
                            position.sold_at = row['Close']

            # Sell MACD
            if row['MACD_Sell_Signal']:
                for position in self.positions:
                    if position.is_active and position.order_type == "LONG":
                        if row['Close'] * (1 - self.comission) >= position.take_profit:
                            self.cash += row['Close'] * (1 - self.comission)
                            position.is_active = False
                            position.sold_at = row['Close']
                        elif row['Close'] * (1 + self.comission) <= position.stop_loss:
                            self.cash += row['Close'] * (1 - self.comission)
                            position.is_active = False
                            position.sold_at = row['Close']

            # Sell RSI
            if row['RSI_Sell_Signal']:
                for position in self.positions:
                    if position.is_active and position.order_type == "LONG":
                        if row['Close'] * (1 - self.comission) >= position.take_profit:
                            self.cash += row['Close'] * (1 - self.comission)
                            position.is_active = False
                            position.sold_at = row['Close']
                        elif row['Close'] * (1 + self.comission) <= position.stop_loss:
                            self.cash += row['Close'] * (1 - self.comission)
                            position.is_active = False
                            position.sold_at = row['Close']
        print("Final Cash Balance:", self.cash)
        #x= self.cash
        print("Open Positions:")
        for position in self.positions:
            print(f"Timestamp: {position.timestamp}, Bought at: {position.bought_at}, Sold at: {position.sold_at}")


    def optimize_strategy(self, parameters):
        sl, tp, *strat_args = parameters
        self.stop_loss = sl
        self.take_profit = tp
        # Set other strategy arguments here if needed

        # Perform backtesting with the updated parameters
        self.backtest()

# Create an instance of the Backtesting class
backtester = Backtesting(trading_data, comission, stop_loss, take_profit)

# Define the optimization function
def minimize_port(params):
    backtester.optimize_strategy(params)
    return -backtester.cash  # You want to maximize the final cash balance

# Parameter optimization 
parameter_bounds = [(0.01, 0.05),  
                    (0.01, 0.05),  
                    (1, 50), 
                    (51, 220)]

# Initial guess for parameters
initial_guess = [0.03, 0.03, 25, 135]  # Adjust these values as needed

# Perform optimization
result = minimize(minimize_port, initial_guess, bounds=parameter_bounds)

# Retrieve optimized parameters
optimal_params = result.x

# Print the optimal parameters and final cash balance
print("Optimal Parameters:", optimal_params)
print("Final Cash Balance:", -result.fun)


In [24]:
# Backtesting
comission = .025
stop_loss = .035
take_profit = .035
cash = 1000000
positions = []
operations = []
trading_data = train_d
class Backtesting:
    def init(self, trading_data, comission, stop_loss, take_profit):
        self.trading_data = trading_data
        self.comission = comission
        self.stop_loss = stop_loss
        self.take_profit = take_profit
        self.cash = 1000000 # Initial cash balance
        self.positions = []


    class Order:
        def __init__(self, timestamp, bought_at, stop_loss,
                     take_profit, order_type, sold_at=None,
                     is_active=True):
            self.timestamp = timestamp
            self.bought_at = bought_at
            self.sold_at = sold_at
            self.stop_loss = stop_loss
            self.take_profit = take_profit
            self.order_type = order_type
            self.is_active = is_active

    def backtest(self):
        for i, row in self.trading_data.iterrows():
            # Close positions
            price = row['Close']
            for position in self.positions:
                if position.is_active:
                    if price <= position.stop_loss:
                        self.cash += price * (1 - self.comission)
                        position.is_active = False
                        position.sold_at = price
                    elif price >= position.take_profit:
                        self.cash += price * (1 - self.comission)
                        position.is_active = False
                        position.sold_at = price

            # Buy SMA
            if row['SMA_Buy_Signal']:
                if self.cash >= row['Close'] * (1 + self.comission):
                    self.cash -= row['Close'] * (1 + self.comission)
                    order = self.Order(timestamp=row['Timestamp'],
                                       bought_at=row['Close'],
                                       stop_loss=row['Close'] * (1 - self.stop_loss),
                                       take_profit=row['Close'] * (1 + self.take_profit),
                                       order_type="LONG")
                    self.positions.append(order)

            # Buy MACD
            if row['MACD_Buy_Signal']:
                if self.cash >= row['Close'] * (1 + self.comission):
                    self.cash -= row['Close'] * (1 + self.comission)
                    order = self.Order(timestamp=row['Timestamp'],
                                       bought_at=row['Close'],
                                       stop_loss=row['Close'] * (1 - self.stop_loss),
                                       take_profit=row['Close'] * (1 + self.take_profit),
                                       order_type="LONG")
                    self.positions.append(order)

            # Buy RSI
            if row['RSI_Buy_Signal']:
                if self.cash >= row['Close'] * (1 + self.comission):
                    self.cash -= row['Close'] * (1 + self.comission)
                    order = self.Order(timestamp=row['Timestamp'],
                                       bought_at=row['Close'],
                                       stop_loss=row['Close'] * (1 - self.stop_loss),
                                       take_profit=row['Close'] * (1 + self.take_profit),
                                       order_type="LONG")
                    self.positions.append(order)

            # Sell SMA
            if row['SMA_Sell_Signal']:
                for position in self.positions:
                    if position.is_active and position.order_type == "LONG":
                        if row['Close'] * (1 - self.comission) >= position.take_profit:
                            self.cash += row['Close'] * (1 - self.comission)
                            position.is_active = False
                            position.sold_at = row['Close']
                        elif row['Close'] * (1 + self.comission) <= position.stop_loss:
                            self.cash += row['Close'] * (1 - self.comission)
                            position.is_active = False
                            position.sold_at = row['Close']

            # Sell MACD
            if row['MACD_Sell_Signal']:
                for position in self.positions:
                    if position.is_active and position.order_type == "LONG":
                        if row['Close'] * (1 - self.comission) >= position.take_profit:
                            self.cash += row['Close'] * (1 - self.comission)
                            position.is_active = False
                            position.sold_at = row['Close']
                        elif row['Close'] * (1 + self.comission) <= position.stop_loss:
                            self.cash += row['Close'] * (1 - self.comission)
                            position.is_active = False
                            position.sold_at = row['Close']

            # Sell RSI
            if row['RSI_Sell_Signal']:
                for position in self.positions:
                    if position.is_active and position.order_type == "LONG":
                        if row['Close'] * (1 - self.comission) >= position.take_profit:
                            self.cash += row['Close'] * (1 - self.comission)
                            position.is_active = False
                            position.sold_at = row['Close']
                        elif row['Close'] * (1 + self.comission) <= position.stop_loss:
                            self.cash += row['Close'] * (1 - self.comission)
                            position.is_active = False
                            position.sold_at = row['Close']
        print("Final Cash Balance:", self.cash)
        #x= self.cash
        print("Open Positions:")
        for position in self.positions:
            print(f"Timestamp: {position.timestamp}, Bought at: {position.bought_at}, Sold at: {position.sold_at}")

In [26]:
backtester = Backtesting(trading_data, comission, stop_loss, take_profit)
backtester.backtest()

TypeError: Backtesting() takes no arguments

In [8]:
# Optimization

def minimize_port(x: np.array, byte_array: List[int]):
    sl, tp, *strat_args = x
    v = backtest(byte_array, sl, tp, strat_args)
    return -v

# Parameter optimization 
parameter_bounds = [(0.01, 0.05),  
                    (0.01, 0.05),  
                    (1, 50), 
                    (51, 220)]

# Initial guess for parameters
initial_guess = [0.03, 0.03, 25, 135]  # Adjust these values as needed

# Perform optimization
result = minimize(minimize_port, initial_guess, args=(combinations,), bounds=parameter_bounds)

# Retrieve optimized parameters
optimal_params = result.x

NameError: name 'minimize' is not defined

In [ ]:
# Optimal Strategy

In [ ]:
# Validation 


In [ ]:
# Passive Strategy


In [ ]:
# Data Visualization

In [ ]:
# Results and conclusions 

In [ ]:
# Documentation 

In [ ]:
#1 Select strategies
#2 Trade Signals(Buy/sell)
#3 Backtesting-> Portafolio Val
#4 Max Profit (SL 1-5%, TP 1-5%, PL1 1-50, PL2 51,200) #Stop loss / take profit 
#5 Params 
#6 Test all possible combinations
#7 Select optimal strategy
#8 Backtest with data